# Calculs sur les consommations de carburant et leur rapport entre carrosserie ou type de carburant

In [85]:
import mysql.connector
import numpy as np
import pandas as pd
import scipy.stats as stats

In [86]:
# Connect to the database
cnx = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='Ecotourisme'
)

# Check if the connection is successful
if cnx.is_connected():
    print('Connected to the database')
else:
    print('Failed to connect to the database')

Connected to the database


In [87]:
query = "SELECT COUNT(*) FROM `CARS` WHERE Energie LIKE 'SUPERETHANOL'"
cursor = cnx.cursor()
cursor.execute(query)
result = cursor.fetchall()
print(result)

[(2,)]


In [88]:
query = "SELECT `Carrosserie`,`Energie`,AVG(`CO2 vitesse mixte Min`/`Conso vitesse mixte Min`/10) AS `facteur emission min`,  AVG(`CO2 vitesse mixte Max`/`Conso vitesse mixte Max`/10) AS `facteur emission max`, AVG(`Conso elec Min`), AVG(`Conso elec Max`) FROM `cars` GROUP BY Carrosserie, Energie ORDER BY `cars`.`Carrosserie` ASC;"
cursor = cnx.cursor()
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=cursor.column_names)
df.pivot_table(index=['Carrosserie'], columns=['Energie'], values=['facteur emission min'], aggfunc=np.mean)

/var/folders/kw/gs_chyyn2bd3x3twksf8gdmr0000gn/T/ipykernel_89243/2251633683.py:6: FutureWarning: The provided callable <function mean at 0x108465f70> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(index=['Carrosserie'], columns=['Energie'], values=['facteur emission min'], aggfunc=np.mean)


facteur emission min                              \
Energie                   ELEC+ESSENC HR ELEC+GAZOLE HR ESS+ELEC HNR   
Carrosserie                                                            
BERLINE                         3.226923            2.9     2.490766   
BREAK                           3.318421            2.9     2.519318   
CABRIOLET                            NaN            NaN     2.483929   
COMBISPACE                           NaN            NaN          NaN   
COUPE                                NaN            NaN     2.494048   
MINIBUS                              NaN            NaN          NaN   
MINISPACE                            NaN            NaN     2.525000   
MONOSPACE                            NaN            NaN     2.416667   
MONOSPACE COMPACT                    NaN            NaN     2.660000   
TS TERRAINS/CHEMINS             3.138462            4.2     2.469862   

                                                                             \
Energie             ESS+G.P.L.   ESSENCE GAZ NAT.VEH GAZ+ELEC HNR    GAZOLE   
Carrosserie                                                                   
BERLINE               2.399333  2.492557        1.96     2.956818  2.947627   
BREAK                 2.600000  2.498983        1.96     3.015476  2.974310   
CABRIOLET                  NaN  2.440105         NaN     2.961667  3.016786   
COMBISPACE            2.466667  2.395238         NaN          NaN  3.002927   
COUPE                      NaN  2.393494         NaN     2.905000  2.991667   
MINIBUS                    NaN       NaN         NaN     2.880000  2.833817   
MINISPACE                  NaN       NaN         NaN          NaN  3.125000   
MONOSPACE                  NaN       NaN         NaN          NaN  2.812024   
MONOSPACE COMPACT     2.333333  2.411373         NaN          NaN  2.895500   
TS TERRAINS/CHEMINS   2.431333  2.443288         NaN     2.886385  2.891489   

                                  
Energie             SUPERETHANOL  
Carrosserie                       
BERLINE                 1.642857  
BREAK                        NaN  
CABRIOLET                    NaN  
COMBISPACE                   NaN  
COUPE                        NaN  
MINIBUS                      NaN  
MINISPACE                    NaN  
MONOSPACE                    NaN  
MONOSPACE COMPACT            NaN  
TS TERRAINS/CHEMINS     1.742857

# Calcul des facteurs d'émission à partir des données de l'ADEME

## Définition de la classe Fuel

In [89]:
class Fuel:

    def __init__(self, name, PCI, MV, combustion):
        self.name = name
        self.PCI = PCI  # PCI in GJ/T
        self.MV = MV    # in kg/m3
        self.combustion = combustion  # in kgCO2eq/GJ

    def __str__(self):
        return f"{self.name}: PCI: {self.PCI} GJ/kg, MV: {self.MV} kg/m3, combustion: {self.combustion} kgCO2eq/GJ"

    def __repr__(self):
        return f"Fuel: {self.name}, PCI: {self.PCI} GJ/kg, MV: {self.MV} kg/m3, combustion: {self.combustion} kgCO2eq/GJ"
    
    # return the mass of CO2eq produced by 1 kg of energy (kgCO2eq/kg)
    def FE_massique(self):
        return self.combustion * self.PCI
    
    # return the volume of CO2eq produced by 1 m3 of energy (kgCO2eq/m3)
    def FE_volumique(self):
        # we need FE_energetique in kgCO2eq/GJ 
        # we need MV in t/m3 but we have it in kg/m3   1t = 1000kg
        return self.FE_massique() * self.MV/1000
    
    # return the volume of CO2eq produced by 1 L of energy (kgCO2eq/L)
    def FE_volumique_par_L(self):
        # we need FE in kgCO2eq/L but we have it in kgCO2eq/m3 and 1m3 = 1000L
        return self.FE_volumique() / 1000

Définition des instances de la classe Fuel en accord avec les données de [la base Empreinte de l'ADEME](https://base-empreinte.ademe.fr/documentation/base-carbone)

In [92]:
fuels_pure = [
            Fuel("Petrole Brut", 42, 900, 73.3),
            Fuel("Fioul domestique", 42, 845, 75.5),
            Fuel("Fioul domestique lourd", 40, 900, 78.6),
            # Fuel("Combustible haute viscosité (CHV)", 39.2, None, None),
            Fuel("Essence", 44, 747, 74.2),
            Fuel("Gasoil", 42.6, 832, 75.6),
            Fuel("Butane", 45.6, 538, 64.8),
            Fuel("Propane", 46, 538, 64.8),
            # Fuel("GPL", 46, 538, None),
            Fuel("Carbureacteur jet B et essence aviation (AvGas)", 44, 800, 71.9),
            Fuel("Essence aviation", 44, 800, 71.9),
            Fuel("Kérosène (Jet A et A1)", 44, 800, 71.9),
            Fuel("Kérosène (Jet A et A1) amont", 44, 800, 86.8)
        ]

for fuel in fuels_pure:
    print(f'{fuel.name}: {fuel.FE_volumique_par_L()}')

Petrole Brut: 2.77074
Fioul domestique: 2.6794949999999997
Fioul domestique lourd: 2.8296
Essence: 2.4388056000000002
Gasoil: 2.67950592
Butane: 1.58972544
Propane: 1.6036704
Carbureacteur jet B et essence aviation (AvGas): 2.5308800000000007
Essence aviation: 2.5308800000000007
Kérosène (Jet A et A1): 2.5308800000000007
Kérosène (Jet A et A1) amont: 3.0553600000000003
